In [102]:
from messari.messari import Messari
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from stockstats import StockDataFrame
import os
import json
import time
import funciones_apis #funciones para importar datos de la api y crear el dataset
import datetime
from twelvedata import TDClient
import pickle


In [ ]:
messari = Messari(api_messari) #Claves api messari 

In [ ]:
indicators= pd.read_json('technical_indicators.json') 
lista_indiadores = list(indicators.index) #indicadores con explciacion de la api

In [ ]:
td = TDClient(apikey=api_twelvedata) #key de api twelvedata


ts = td.time_series(symbol=symbol,interval=interval,timezone="America/New_York",start_date = start,end_date=end
                        ,order=order,outputsize=outputsize,exchange=exchange)

df = ts.as_pandas() #df con precio e indicadores técnicos

In [ ]:
os.chdir(r'C:\Users\Enrique\Desktop\Proyecto\Datasets')
df.to_csv('Precio 2012 - 2022 bitfinex.csv', header=True, index=True) #saved to csv file

In [ ]:
dfit = api_twelve("BTC/USD", "1day", '2012-01-31','2022-07-09', 5000,  'asc', 'Bitfinex')

In [ ]:
dfit.to_csv('Precio y IT 2012 - 2022.csv',  index= False) #saved to csv file

In [136]:
td = TDClient(apikey=api_twelvedata) # scamos df con el precio del NASDAQ para comprobar correlacion

# Construct the necessary time series
ts = td.time_series(
    symbol="GSPC",
    interval="1day",
    start_date = '2012-01-31',
    end_date = '2022-07-09',
    outputsize=5000,
    timezone="America/New_York")

dfsp500 = ts.as_pandas()
dfsp500 = dfsp500.drop_duplicates()

In [137]:
dfsp500.shape

(2686, 5)

In [138]:
td = TDClient(apikey=api_twelvedata) # scamos df con el precio del sp500 para comprobar correlacion

# Construct the necessary time series
ts = td.time_series(
    symbol="IXIC",
    interval="1day",
    start_date = '2012-01-31',
    end_date = '2022-07-09',
    outputsize=5000,
    timezone="America/New_York")

dfnasdaq = ts.as_pandas()
dfnasdaq = dfnasdaq.drop_duplicates()

In [139]:
dfnasdaq.shape

(2685, 5)

In [140]:
td = TDClient(apikey=api_twelvedata)

#sacamos df con el precio del dxy para comprobar correlacion
ts = td.time_series(
    symbol="DXY",
    interval="1day",
    start_date = '2012-01-31',
    end_date = '2022-07-09',
    outputsize=5000,
    timezone="America/New_York")

dfdxy = ts.as_pandas()
dfdxy = dfdxy.drop_duplicates()

In [141]:
dfdxy.shape

(2626, 5)

In [142]:
dfsp500.drop("volume", axis = 1, inplace = True)
dfnasdaq.drop("volume", axis = 1, inplace = True)
dfdxy.drop("volume", axis = 1, inplace = True)

#unimos los 3 dataframes
dfindex = pd.merge(dfsp500,dfnasdaq, how = "left", left_index = True, right_index = True,)
dfindex = pd.merge(dfindex,dfdxy, how = "left", left_index = True, right_index = True,)
dfindex.columns = ("open_sp", "high_sp", "low_sp", "close_sp","open_nq", "high_nq", "low_nq", "close_nq","open_dxy", "high_dxy", "low_dxy", "close_dxy")

In [145]:
dfindex.shape

(2801, 12)

In [144]:
dfindex = dfindex.drop_duplicates() #quitamos registros duplicados

In [146]:
dfindex.to_csv('Indices bursátiles.csv') #saved to csv

In [ ]:
os.chdir(r'C:\Users\Enrique\Desktop\Proyecto\Documentación apis')

f = open('metrics messari.json', encoding = 'utf8') #json con las métricas on chain y su informacion
metricas = json.load(f)

#f.close()

In [ ]:
lista_metricas = []

for i in metricas["data"]["metrics"]:
    x = []
    x.append(i["metric_id"])
    x.append(i["name"])
    x.append(i["description"])
    lista_metricas.append(x)
    
lista_idmetrics = []
for i in lista_metricas:
    lista_idmetrics.append(i[0])
    
lista_idmetrics_sinerc = [] 
lista_indices = [5,9,54,69,86,100,140,161,177,209,212,215,218,225,230,239] 
#eliminamos las métricas que no tiene que ver con bitcoin
for i in lista_idmetrics:
    if lista_idmetrics.index(i) not in lista_indices:
        lista_idmetrics_sinerc.append(i)


In [ ]:
dfocm = api_messarri() #duncion creada que coge todas lás ocm desde 2012 a 2022 y devuelve en dataframe

In [ ]:
dfocm.to_csv('Metricas ocm.csv',  index= False) 

In [ ]:
dfit.index = dfit["datetime"].map(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d"))
dfocm.index = dfocm["Unnamed: 0"].map(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d"))
dfit.drop('datetime', axis =1, inplace = True)
dfocm.drop("Unnamed: 0", axis =1, inplace = True)
dfit_ocm = dfit.merge(dfocm, how = "inner", left_index = True, right_index = True)
#unimos los dataframes d eindicadores y métricas por la fecha

In [ ]:
dfit_ocm["Change percentage"] = dfit_ocm["Close"].pct_change()*100 #creamos la variable de porcentaje de cambio

In [ ]:
dfit_ocm.to_csv('Precio-it-ocm.csv',  index= False) 

In [ ]:
dffinal = pd.merge(dfit_ocm,dfindex, how = 'left', left_index = True, right_index = True)
dffinal["Change percentage"] = dffinal["Close"].pct_change()*100

In [153]:
dffinal.to_csv('Dataframe final.csv',  index= False) #dataframe final sin preprocesamiento

In [155]:
dflag = dffinal.copy()

In [156]:
lista_columnas = list(dflag.columns) #creamos las varibales laggeadas

for i in range(1,9):
    for j in lista_columnas:
        dflag[j+'_'+str(i)] = dflag[j].shift(i)

C:\Users\Enrique\AppData\Local\Temp/ipykernel_60912/4279602137.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dflag[j+'_'+str(i)] = dflag[j].shift(i)


In [157]:
dflag.shape

(3794, 3276)

In [158]:
dflag.to_csv('Dataframe final lag.csv', index= False)

In [4]:
dflag.dropna(axis=1, thresh=800, inplace = True) #eliminamos columnas con muchos nans

In [5]:
dflag = dflag.drop(dflag.columns[[334,682,1030,1378,1726,2074,2422,2770,3118]], axis='columns') #eliminamos columnas que no nos sirven

In [85]:
df.to_csv('Dataframe final lag sin nans.csv',index= False)

In [78]:
os.chdir(r'C:\Users\Enrique\Desktop\Proyecto\Datasets')
df = pd.read_csv('Dataframe final lag sin nans.csv')

In [147]:
fh = open('Dataframe pickle.pkl','wb') #guardamos el df en formato pickle
pickle.dump(df,fh)

In [38]:
nulos = df.isnull().sum()
nulos = nulos[nulos>500]
df.drop(nulos.index, axis=1, inplace = True)

In [ ]:
df.drop(['Hyperbolic Cosine_1','Hyperbolic Cosine_2','Hyperbolic Cosine_3','Hyperbolic Cosine_4',
                'Hyperbolic Cosine_5','Hyperbolic Cosine_6','Hyperbolic Cosine_7','Hyperbolic Cosine_8'],inplace = True, axis = 1)
#eliminamo estas columnas yq que son valores infinitos y nos causan problemas en los modelos

In [146]:
df.to_csv('Dataframe final lag sin nans.csv')

In [51]:
nulos = df['2014':].isnull().sum()
nulos = nulos[nulos>0]
len(nulos)
#eliminamos el resto de columnas con algun nan

36

In [52]:
df.drop(nulos.index, axis=1, inplace = True)

In [54]:
df = pd.read_csv('Dataframe final lag sin nans.csv')

In [57]:
nulos = df['2014':].isnull().sum()
nulos = nulos[nulos>0]
len(nulos)

0

In [61]:
 df.drop(['Unnamed: 0_1','Unnamed: 0_2','Unnamed: 0_3','Unnamed: 0_4','Unnamed: 0_5','Unnamed: 0_6','Unnamed: 0_7','Unnamed: 0_8'], inplace = True, axis = 1)

In [95]:
count = np.isinf(df).values.sum()
count

2306

In [96]:
col_name = df.columns.to_series()[np.isinf(df).any()]
print(col_name) #tienen infs

%B Indicator              %B Indicator
Hyperbolic Cosine    Hyperbolic Cosine
%B Indicator_1          %B Indicator_1
%B Indicator_2          %B Indicator_2
%B Indicator_3          %B Indicator_3
%B Indicator_4          %B Indicator_4
%B Indicator_5          %B Indicator_5
%B Indicator_6          %B Indicator_6
%B Indicator_7          %B Indicator_7
%B Indicator_8          %B Indicator_8
dtype: object


In [98]:
df.drop(col_name, axis = 1, inplace = True) #eliminamos columnas con valores infinitos

In [139]:
df = pd.read_csv('Dataframe final lag sin nans.csv')

In [122]:
#creamos funcion para  la variable objetivo de 12 clases

def rangos(i): 
    if i < -5: return '<-5%'
    if i >= -5 and i < -4: return '-5/-4%'
    if i >= -4 and i < -3: return '-4/-3%'
    if i >= -3 and i < -2: return '-3/-2%'
    if i >= -3 and i < -2: return '-3/-2%'
    if i >= -2 and i < -1: return '-2/-1%'
    if i >= -1 and i < 0: return '-1/-0%'
    if i >= 0 and i < 1: return '0/1%'
    if i >= 1 and i < 2: return '1/2%'
    if i >= 2 and i < 3: return '2/3%'
    if i >= 3 and i < 4: return '3/4%'
    if i >= 4 and i < 5: return '4/5%'
    if i >= 5 : return '>5%'

In [133]:
#creamos funcion para variable objetivo de 3 clases

def rangos3(i): 
    if i < -1: return -1
    if i >= -1 and i <= 1: return 0
    if i >= 1 : return 1

In [123]:
df["Range"] = df["Change percentage"].apply(rangos)

In [128]:
df.insert(329, "Range", insertar)

In [136]:
x = df["Change percentage"].apply(rangos3)

In [144]:
df.insert(330, "Range3", x)

In [143]:
df.columns[331]

'Open_1'

In [145]:
df[["Change percentage", "Range", "Range3"]]

,Change percentage,Range,Range3
0,NaN,NaN,NaN
1,0.328947,0/1%,0.0
2,-2.295082,-3/-2%,-1.0
3,-1.510067,-2/-1%,-1.0
4,-3.066440,-4/-3%,-1.0
...,...,...,...
3789,-0.271954,-1/-0%,0.0
3790,1.924016,1/2%,1.0
3791,5.170708,>5%,1.0
3792,-0.092507,-1/-0%,0.0


In [149]:
df["Range3"].value_counts(normalize=True)

 1.0    0.361192
 0.0    0.348800
-1.0    0.290008
Name: Range3, dtype: float64